In [27]:
# EV Charging Load Profile Generator
"""
Project: EV Fleet Charging Profile Generator
Purpose:
- Generate EV truck fleet charging load profiles for Gridcog input
- Visualise per-truck charging behaviour with charger constraints
- Support user-configurable fleet size, truck mix, mid-shift charging, and charger limits

Author: Volta Chief AI and Data Officer
Version: 0.3 (Stable)
Last Updated: 04 April 2025
Environment: Google Colab
"""

# --- Install & Imports ---
!pip install xlsxwriter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from io import BytesIO
from IPython.display import display
import ipywidgets as widgets
from tqdm.notebook import tqdm
import time

# --- Truck Specifications ---
TRUCK_MODELS = {
    'Volvo FM electric': {'battery_kWh': 540, 'useful_kWh': 400, 'max_charge_kW': 250, 'efficiency_kWh_per_km': 1.333},
    'Windrose': {'battery_kWh': 729, 'useful_kWh': 705, 'max_charge_kW': 960, 'efficiency_kWh_per_km': 1.052},
    'Scania 624 kWh': {'battery_kWh': 624, 'useful_kWh': 600, 'max_charge_kW': 375, 'efficiency_kWh_per_km': 1.091}
}

CHARGING_EFFICIENCY = 0.94
LOOP_DISTANCE_KM = 200
LOOPS_PER_SHIFT = 2
ENERGY_PER_SHIFT = {
    name: round(LOOP_DISTANCE_KM * LOOPS_PER_SHIFT * spec['efficiency_kWh_per_km'] / CHARGING_EFFICIENCY, 2)
    for name, spec in TRUCK_MODELS.items()
}

# --- UI Widgets ---
fleet_size_slider = widgets.IntSlider(value=20, min=20, max=50, description='Fleet Size:')
truck_mix_dropdown = widgets.Dropdown(
    options=['50% Volvo / 50% Windrose', '100% Volvo', '100% Windrose', 'Mixed: add Scania'],
    value='50% Volvo / 50% Windrose', description='Fleet Mix:')
start_time_text = widgets.Text(value='06:00', description='Start Time:')
stagger_interval_slider = widgets.IntSlider(value=7, min=1, max=30, step=1, description='Stagger (min):')
charger_limit_slider = widgets.IntSlider(value=50, min=1, max=50, description='Max Chargers:')
mid_shift_toggle = widgets.Checkbox(value=True, description='Mid-shift Charging')
mid_shift_models_selector = widgets.SelectMultiple(
    options=list(TRUCK_MODELS.keys()), value=['Volvo FM electric'], description='Mid-shift Models'
)
performance_mode = widgets.ToggleButtons(options=['Detailed', 'Daily Average'], description='Output Mode:')
night_shift_toggle = widgets.Checkbox(value=False, description='Night Shift')
start_date_picker = widgets.DatePicker(description='Start Date:', value=date.today())
end_date_picker = widgets.DatePicker(description='End Date:', value=date.today())
run_button = widgets.Button(description="Run Simulation")
status_output = widgets.Output()

display(widgets.VBox([
    fleet_size_slider, truck_mix_dropdown, start_time_text, stagger_interval_slider,
    charger_limit_slider, mid_shift_toggle, mid_shift_models_selector,
    night_shift_toggle, performance_mode, start_date_picker, end_date_picker, run_button, status_output
]))

# --- Schedule Generator ---
def generate_fleet_schedule(fleet_size, truck_mix, start_time_str, stagger_min, include_night):
    if truck_mix == '50% Volvo / 50% Windrose':
        mix = ['Volvo FM electric'] * (fleet_size // 2) + ['Windrose'] * (fleet_size - fleet_size // 2)
    elif truck_mix == '100% Volvo':
        mix = ['Volvo FM electric'] * fleet_size
    elif truck_mix == '100% Windrose':
        mix = ['Windrose'] * fleet_size
    elif truck_mix == 'Mixed: add Scania':
        mix = ['Volvo FM electric'] * (fleet_size // 3) + ['Windrose'] * (fleet_size // 3)
        mix += ['Scania 624 kWh'] * (fleet_size - len(mix))

    base_start = datetime.strptime(start_time_str, '%H:%M')
    schedule = []
    for i, truck_type in enumerate(mix):
        dep = base_start + timedelta(minutes=i * stagger_min)
        arr = dep + timedelta(hours=12)
        lunch = dep + timedelta(hours=6)
        schedule.append({
            'TruckID': f'Truck_{i+1}', 'TruckType': truck_type,
            'Departure': dep, 'Arrival': arr, 'LunchBreak': lunch,
            'Energy_kWh': ENERGY_PER_SHIFT[truck_type]
        })

    if include_night:
        night = []
        for row in schedule:
            night.append({
                'TruckID': row['TruckID'] + '_N',
                'TruckType': row['TruckType'],
                'Departure': row['Departure'] + timedelta(hours=12),
                'Arrival': row['Arrival'] + timedelta(hours=12),
                'LunchBreak': row['LunchBreak'] + timedelta(hours=12),
                'Energy_kWh': row['Energy_kWh']
            })
        schedule.extend(night)

    return pd.DataFrame(schedule)

# --- Charging Simulation ---
def simulate_charging(schedule_df, interval_mins=30, max_chargers=50, mid_shift=False):
    schedule_df[['Arrival', 'Departure', 'LunchBreak']] = schedule_df[['Arrival', 'Departure', 'LunchBreak']].apply(pd.to_datetime)

    start_time = schedule_df['Departure'].min().replace(hour=0, minute=0)
    end_time = (schedule_df['Arrival'].max() + timedelta(hours=12)).ceil(f'{interval_mins}min')
    time_index = pd.date_range(start=start_time, end=end_time, freq=f'{interval_mins}min')

    truck_ids = schedule_df['TruckID'].unique().tolist()
    charging_matrix = pd.DataFrame(0.0, index=time_index, columns=truck_ids)
    charger_usage = {t: 0 for t in time_index}

    for _, row in schedule_df.iterrows():
        truck_id = row['TruckID']
        truck_type = row['TruckType']
        energy_needed = row['Energy_kWh']
        max_kW = TRUCK_MODELS[truck_type]['max_charge_kW']
        slot_kWh = max_kW * (interval_mins / 60)

        delivered = 0
        windows = []
        if mid_shift and truck_type in mid_shift_models_selector.value:
            windows.append((row['LunchBreak'], row['LunchBreak'] + timedelta(hours=1.5)))
        windows.append((row['Arrival'], end_time))

        for start, end in windows:
            now = pd.to_datetime(start).floor(f'{interval_mins}min')
            end = pd.to_datetime(end).ceil(f'{interval_mins}min')
            while now < end and delivered < energy_needed:
                if now in charging_matrix.index and charger_usage.get(now, 0) < max_chargers:
                    charging_matrix.at[now, truck_id] = round(max_kW / 1000, 2)
                    charger_usage[now] += 1
                    delivered += slot_kWh
                now += timedelta(minutes=interval_mins)
            if delivered >= energy_needed:
                break

    summary = pd.DataFrame(index=time_index)
    summary['MW'] = charging_matrix.sum(axis=1).round(2)
    summary['No. charging'] = (charging_matrix > 0).sum(axis=1)
    kWh_matrix = charging_matrix * (interval_mins / 60)
    kWh_matrix.columns = [col + '_kWh' for col in kWh_matrix.columns]

    return pd.concat([summary, charging_matrix, kWh_matrix], axis=1)

# --- Simulation Trigger ---
def on_run_clicked(b):
    with status_output:
        status_output.clear_output()
        print("Running simulation...")

        df_schedule = generate_fleet_schedule(
            fleet_size_slider.value, truck_mix_dropdown.value,
            start_time_text.value, stagger_interval_slider.value,
            night_shift_toggle.value
        )

        all_days = pd.date_range(start=start_date_picker.value, end=end_date_picker.value, freq='D')
        full_schedule = []
        for day in all_days:
            temp = df_schedule.copy()
            for t in ['Departure', 'Arrival', 'LunchBreak']:
                temp[t] = temp[t].apply(lambda dt: pd.Timestamp.combine(day, dt.time()))
            full_schedule.append(temp)

        df_schedule = pd.concat(full_schedule, ignore_index=True)
        output_df = simulate_charging(
            df_schedule,
            max_chargers=charger_limit_slider.value,
            mid_shift=mid_shift_toggle.value
        )

        if performance_mode.value == 'Daily Average':
            output_df = output_df[['MW', 'No. charging']].resample('D').mean()

        output_file = generate_outputs_csv(output_df)

        fig, ax1 = plt.subplots(figsize=(14, 5))
        output_df[['MW']].plot(ax=ax1, color='tab:blue')
        ax2 = ax1.twinx()
        output_df[['No. charging']].plot(ax=ax2, color='tab:orange', linestyle='--')
        ax1.set_ylabel('MW Load')
        ax2.set_ylabel('Chargers Active')
        ax1.set_title('Fleet Charging Load & Charger Utilisation')
        ax1.grid(True)
        fig.tight_layout()
        plt.show()

        from google.colab import files
        with open("charging_output_matrix.csv", "wb") as f:
            f.write(output_file.read())
        files.download("charging_output_matrix.csv")
        print("✅ Simulation complete. File downloaded.")

run_button.on_click(on_run_clicked)

# --- Output Helper ---
def generate_outputs_csv(df):
    out = BytesIO()
    df.to_csv(out)
    out.seek(0)
    return out
